# Token Embeddings using gensim 

In [1]:
!pip install gensim

In [2]:
import gensim.downloader as api
model = api.load("word2vec-google-news-300")

In [3]:
word_vector = model
print(word_vector['hello','cat','laptop','dog'])

[[-0.05419922  0.01708984 -0.00527954 ... -0.36523438 -0.13769531
  -0.12890625]
 [ 0.0123291   0.20410156 -0.28515625 ...  0.4921875  -0.27539062
   0.05615234]
 [ 0.02648926 -0.1640625  -0.00701904 ...  0.0859375  -0.31835938
   0.03125   ]
 [ 0.05126953 -0.02233887 -0.17285156 ...  0.41601562 -0.35546875
   0.22265625]]


In [4]:
print(word_vector['cat'].shape)

(300,)


### Similar Words

### King + Woman - Man = ?


In [5]:
print(word_vector.most_similar(positive=['king','woman'],negative=['man'],topn=3))

[('queen', 0.7118193507194519), ('monarch', 0.6189674735069275), ('princess', 0.5902431011199951)]


### Create Token Embeddings

In [6]:
import torch

In [7]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size,output_dim)

In [8]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [9]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


# Embedding and positional embedding from scratch

In [10]:
import tiktoken

In [11]:
with open("the-verdict.txt", "r") as f:
    text = f.read()

In [12]:
import torch
from torch.utils.data import DataLoader, Dataset

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Create overlapping sequences using sliding window
        # stride controls overlap: stride=1 = maximum overlap, stride=max_length = no overlap
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i+max_length]
            # Target is input shifted by 1 position (next token prediction)
            target_chunk = token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [13]:
def create_dataloader_v1(txt, batch_size, max_length, stride,
                         shuffle, drop_last=True, num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    
    # drop_last=True prevents loss spikes from incomplete batches
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader


In [14]:
max_length = 4 
batch_size=8

dataloader = create_dataloader_v1(
    text, batch_size=batch_size, max_length=max_length,
    stride=4, shuffle=False
)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [15]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [16]:
print("Targer IDs:\n", targets)
print("\nInputs shape:\n", targets.shape)

Targer IDs:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])

Inputs shape:
 torch.Size([8, 4])


## Understanding Embedding Layer Dimensions

**Key Concept:** `vocab_size = 50257` does NOT mean we're processing 50257 tokens!

- **`vocab_size = 50257`**: This is the size of the embedding lookup table. The embedding layer has a weight matrix of shape `[50257, 256]` - one 256-dimensional vector for each possible token ID (0 to 50256).

- **`inputs` shape `[8, 4]`**: 
  - 8 = batch size (8 sequences)
  - 4 = sequence length (4 tokens per sequence)
  - Total: 32 token IDs are being processed

- **How it works**: Each token ID in `inputs` is used as an index to look up its corresponding embedding vector from the 50257×256 lookup table.

- **Output shape `[8, 4, 256]`**:
  - 8 = batch size (same as input)
  - 4 = sequence length (same as input)
  - 256 = embedding dimension (each token ID is replaced with its 256-dimensional vector)

**Example**: If `inputs[0, 0] = 40`, the embedding layer looks up row 40 from its weight matrix and returns that 256-dimensional vector.


In [17]:
vocab_size = 50257
output_dim = 256

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

token_embedding = embedding_layer(inputs)
token_embedding.shape

torch.Size([8, 4, 256])

### Positional Embedding Layer

In [18]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length,output_dim)


In [19]:
pos_embedding = pos_embedding_layer(torch.arange(max_length))
print(pos_embedding.shape)

torch.Size([4, 256])


##### Adding Token Embeddings to Positional Embeddings

## Key Points:

1. **Token Embeddings**: Represent the semantic meaning of each token
   - Shape: `[batch_size, seq_len, embed_dim]` = `[8, 4, 256]`
   - Each token ID is mapped to a learnable vector

2. **Positional Embeddings**: Represent the position of each token in the sequence
   - Shape: `[seq_len, embed_dim]` = `[4, 256]`
   - Same embedding vector is used for the same position across all sequences

3. **Combining**: Element-wise addition
   - Use `unsqueeze(0)` to add batch dimension: `[4, 256]` → `[1, 4, 256]`
   - PyTorch automatically broadcasts to `[8, 4, 256]` when adding
   - Result: Each token's embedding = its semantic meaning + its position information

**Alternative approach** (without unsqueeze):
```python
# PyTorch broadcasting works automatically:
combined_embedding = token_embedding + pos_embedding  # Works due to broadcasting
```


In [20]:
# Get token embeddings for the batch
token_embedding = embedding_layer(inputs)
print("Token embedding shape:", token_embedding.shape)

# Get positional embeddings for all positions
pos_indices = torch.arange(max_length)
pos_embedding = pos_embedding_layer(pos_indices)
print("Positional embedding shape:", pos_embedding.shape)

# Add token embeddings to positional embeddings
# pos_embedding needs to be broadcasted to match batch dimension
# Shape: [4, 256] -> [1, 4, 256] -> [8, 4, 256] (broadcasted)
combined_embedding = token_embedding + pos_embedding.unsqueeze(0)
print("Combined embedding shape:", combined_embedding.shape)
print("\nFirst token embedding:", token_embedding[0, 0, :5])
print("First positional embedding:", pos_embedding[0, :5])
print("First combined embedding:", combined_embedding[0, 0, :5])


Token embedding shape: torch.Size([8, 4, 256])
Positional embedding shape: torch.Size([4, 256])
Combined embedding shape: torch.Size([8, 4, 256])

First token embedding: tensor([-0.0640,  0.3317,  0.1070, -0.1428, -0.3014], grad_fn=<SliceBackward0>)
First positional embedding: tensor([ 0.5423, -0.1224, -1.4150, -0.3142,  0.2827], grad_fn=<SliceBackward0>)
First combined embedding: tensor([ 0.4784,  0.2094, -1.3080, -0.4570, -0.0187], grad_fn=<SliceBackward0>)
